In [1]:
import sys
sys.executable


'/root/home/envforir/bin/python'

In [2]:
# # 데몬 인스턴스 만들기 
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

In [3]:
es_username = 'elastic'
es_password = 'XrKWVzZkg-R==V9bxHoy'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/upstage-ai-final-ir2/HM/elasticsearch-8.8.0/config/certs/http_ca.crt", verify_certs=False)
# CA 인증서 경로 지정

# ElasticSearch Client 정보 확인 
es.info()

/root/home/envforir/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
# 데몬 구동 확인 
!ps -ef | grep elasticsearch

iruser    740812       1  0 Jun25 ?        00:05:35 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-5356112457491983244 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupancyPercent=30 

In [5]:
# from sentence_transformers import SentenceTransformer

# # Sentence Transformer 모델 초기화(한국어 임베딩 생성 가능한 어떤 모델도 가능)
# model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")   
# # 이 모델을 사용하여 doc['content']를 임베딩 

# # Sentence Transformer를 이용하여 임베딩 생성
# def get_embedding(sentences):
#     return model.encode(sentences)

# # 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
# def get_embeddings_in_batches(docs, batch_size=100):
#     batch_embeddings = []
#     for i in range(0, len(docs), batch_size):
#         batch = docs[i: i+batch_size]
#         contents = [doc["content"] for doc in batch]
#         embeddings = get_embedding(contents)    # doc의 "content" 부분을 임베딩 
#         batch_embeddings.extend(embeddings)
#         print(f'batch {i}')
#     return batch_embeddings

In [6]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = [get_embedding(content) for content in contents]
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings

In [7]:
# 새로운 index 생성 
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):   # 만약 인덱스가 이미 존재한다면 
        es.indices.delete(index=index)   # 새로운 것으로 설정하기 위해 삭제 
    es.indices.create(index=index, settings=settings, mappings=mappings)  # 지정된 settings로 새로운 인덱스 생성 

# 지정된 인덱스 삭제 
def delete_es_index(index):
    es.indices.delete(index=index)
    
# ElasticSearch 헬퍼 함수를 사용하여 대량 인덱싱 수행 
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    return helpers.bulk(es, actions)

### 검색함수

In [8]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 


# dense_retrieve: Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]   # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기 
    knn = {"field": "embeddings",                     # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정 
           "query_vector": query_embedding.tolist(),
           "k": size,    # k: 검색할 최근접 이웃의 개수 
           "num_candidates": 100}  # KNN 검색에 사용할 후보 문서 개수 
    return es.search(index='test', knn=knn)           # 지정된 인덱스에서 벡터 유사도 검색 수행 

# dense_retrieve_2: sparse_retrieve 함수로 필터링된 문서들에 대해서만 유사도 검색
def dense_retrieve_2(query_str, documents, size):
    query_embedding = get_embedding([query_str])[0]
    document_ids = [doc['_id'] for doc in documents]
    
    # docid와 _id의 매핑 생성
    id_to_docid = {doc['_id']: doc['_source']['docid'] for doc in documents}

    body = {
        "_source": {"includes": ["content", "embeddings"]},
        "query": {
            "bool": {
                "must": [
                    {
                        "ids": {
                            "values": document_ids
                        }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                                "params": {"query_vector": query_embedding.tolist()}
                            }
                        }
                    }
                ]
            }
        }
    }

    results = es.search(index='test', body=body, size=size)
    
    # 결과를 스코어에 따라 정렬하고 docid 추가
    sorted_results = sorted(results['hits']['hits'], key=lambda x: x['_score'], reverse=True)
    for result in sorted_results:
        result['_source']['docid'] = id_to_docid[result['_id']]
    
    return {'hits': {'hits': sorted_results[:size]}}

In [9]:
# 색인을 위한 settings 설정 
settings = { "analysis": {"analyzer": {"nori": {"type": "custom",  #>> custom: 사용자 정의 분석기임을 명시 
                                                "tokenizer": "nori_tokenizer",
                                                "decompound_mode": "mixed",  # 복합 명사를 분해하는 방식을 설정. mixed: 복합명사를 분해하되, 분해된 토큰과 원본 토큰을 모두 저장 
                                                "filter": ["nori_posfilter"]}},
                          "filter": {"nori_posfilter":{"type": "nori_part_of_speech",
                                                       "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]}}}  # 제거할 품사 태그를 지정 
            }

# 색인을 위한 mapping 설정(역색인 필드, 임베딩 필드 모두 설정)
mappings = { "properties": {"content": {"type": "text",
                                        "analyzer": "nori"},
                            "embeddings": {"type": "dense_vector",
                                           "dims": 768,
                                           "index": True,
                                           "similarity": "l2_norm"}}  # 벡터 간 유사도 계산 방식: L2 norm 
            }


In [10]:
# 위에서 설정한 내용으로 'test' 인덱스 생성 
create_es_index("test", settings, mappings)

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 임베딩: 문서의 content 필드 

In [31]:
# # 문서의 content 필드에 대한 임베딩 생성 
# index_docs = []
# with open("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl") as f:
#     docs = [json.loads(line) for line in f]
# embeddings = get_embeddings_in_batches(docs)

# # 생성한 임베딩을 색인할 필드로 추가
# for doc, embedding in zip(docs, embeddings):
#     doc["embeddings"] = embedding.tolist()
#     index_docs.append(doc)

# # 'test' 인덱스에 대량 문서 추가
# ret = bulk_add("test", index_docs)

# # 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
# print(ret)


In [11]:
# 임베딩: OpenAI 모델

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# OpenAI API를 사용하여 임베딩 생성
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i: i+batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = [get_embedding(content) for content in contents]
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings

# 문서의 content 필드에 대한 임베딩 생성 
index_docs = []
with open("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


BulkIndexError: 500 document(s) failed to index.

### 예제로 확인

In [ ]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "직류와 교류 전류의 차이에 대해 알려줘"

In [ ]:
sparse_results = sparse_retrieve(test_query, 30)

print("Sparse retrieve results:")
for rst in sparse_results['hits']['hits']:
    print('ID:', rst['_source']['docid'], 'score:', rst['_score'], 'source:', rst['_source']['content'][:100])

Sparse retrieve results:


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
# dense_retrieve_2
dense_results = dense_retrieve_2(test_query, sparse_results['hits']['hits'], 3)

print("\nDense retrieve 2 results:")
for rst in dense_results['hits']['hits']:
    print('ID:', rst['_source']['docid'], 'score:', rst['_score'], 'source:', rst['_source']["content"][:100])

/tmp/ipykernel_741802/2743210528.py:51: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index='test', body=body, size=size)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Dense retrieve 2 results:
ID: 144f5e5e-8069-425f-80b3-6388195ba4ee score: 2.3782012 source: 전기 회로에서 DC와 AC의 차이점은 다음과 같다. DC는 직류 전류를 의미하며, 전하가 한 방향으로 흐른다는 것이 특징이다. 이는 전류의 방향이 일정하게 유지되는 것을 의미한다.
ID: de7c0ca2-10d2-4ff1-820a-888a91028edd score: 2.352162 source: 힘-전류 유사성은 전기 회로에서 변위(x)와 전류(I) 사이의 관계를 설명하는 개념입니다. 변위(x)에 대한 전기적 아날로그량은 선속(velocity)입니다. 선속은 단위 시간당 
ID: 97ea3f60-5704-412f-b22d-e5d19eb3609b score: 2.3335586 source: 전구가 병렬로 연결된 회로에서는 각 전구의 전류는 독립적으로 흐르게 됩니다. 따라서 한 전구의 전류가 2A라고 해서 다른 전구의 전류를 정확히 알 수는 없습니다. 전구의 전류는 전


In [14]:
# desnse_retrieve
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.0029772597 source: 아칸소주를 통과하는 물새 수의 매년 변화를 알아보기 위해서는 다음과 같은 방법을 사용할 수 있습니다. 첫째로, 일 년 중 하루를 선택하여 이동하는 철새의 수를 조사합니다. 둘째로, 주 전역의 서로 다른 10개의 호수에서 새의 수를 셉니다. 이를 10년 동안 매년 같은 날 실시하여 데이터를 수집합니다. 이 방법을 통해 아칸소주를 통과하는 물새 수의 매년 변화를 정확하게 파악할 수 있습니다. 이러한 정보는 철새의 이동 패턴과 서식지 변화 등을 이해하는 데 도움이 될 것입니다.
score: 0.0029594307 source: 다윈은 생물의 적응과 변형에 대한 연구를 통해 현대 생물학의 기초를 마련했습니다. 그는 갈라파고스 제도에서 수집한 다양한 종의 동물들을 관찰하고, 이들이 특정 환경에 적응하면서 어떻게 변형을 겪는지를 연구했습니다. 만약 HMS 비글이 갈라파고스 제도를 완전히 우회했다면, 다윈은 이러한 적응과 변형에 대한 중요한 인사이트를 얻지 못했을 것입니다. 다윈의 연구는 인구의 능력을 보여주었는데, 특정 환경에 적응하면서 생물들은 변형을 겪을 수 있는 능력을 갖고 있다는 것을 밝혀냈습니다. 이는 생물의 진화와 다양성을 이해하는 데에 중요한 역할을 하였습니다. 다윈의 연구는 현대 생물학의 기초를 이루는 중요한 이론 중 하나로 여겨지고 있습니다.
score: 0.002955311 source: 인터넷은 현대 사회에서 많은 사람들의 삶을 개선하는 데 큰 도움을 주고 있습니다. 예전에는 정보를 얻기 위해서는 도서관이나 전문가의 도움을 받아야 했지만, 인터넷의 등장으로 사람들은 다양한 위치에서 정보에 접근할 수 있게 되었습니다. 이제는 인터넷을 통해 검색 엔진을 이용하여 원하는 정보를 쉽고 빠르게 찾을 수 있습니다. 또한, 인터넷을 통해 온라인 강의나 교육 자료를 제공받을 수 있어 학습의 기회를 더욱 확장시킬 수 있습니다. 인터넷은 또한 소셜 미디어를 통해 사람들이 서로 소통하고 정보를 공유할 수 있는 플랫폼을 제

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# RAG 구현
준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [15]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "sk-proj-FCdLR9ZMWFm8vjGzk3p3T3BlbkFJi3Dmulq4Tl3lOhZpHMoK"
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI()

# 사용할 모델을 설정
llm_model = "gpt-4-turbo"


In [16]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
당신은 사용자의 질문을 분석하여 검색에 적합한 쿼리를 생성하는 AI 어시스턴트입니다.
사용자의 질문을 받으면, 다음 단계를 따라주세요:
1. 질문의 핵심 주제와 의도를 파악하세요.
2. 검색에 효과적인 키워드를 추출하세요.
3. 추출한 키워드를 사용해 간결하고 명확한 검색 쿼리를 만드세요.
4. 만든 쿼리를 'standalone_query' 파라미터로 'search' 함수에 전달하세요.

예시:
사용자 질문: "금성이 밝게 보이는 이유가 뭐야?"
생성할 standalone_query: "금성 밝기 원인"

항상 검색 쿼리를 생성하고 'search' 함수를 호출해야 합니다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "검색 쿼리를 사용하여 관련 문서를 찾습니다.",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "검색에 사용할 쿼리 문자열"
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [17]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
        model=llm_model,
        messages=msg,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "search"}},  # standalone_query를 잘 추출하지 못함 -> LLM이 반드시 'search' 함수를 사용하도록 강제 
        temperature=0,
        seed=1,
        timeout=10
)
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # retriever 설정!!!
        # 1. sparse_retrieve로 20개 검색
        sparse_results = sparse_retrieve(standalone_query, 20)
        
        # 2. dense_retrieve_2로 최종 3개 선정
        search_result = dense_retrieve_2(standalone_query, sparse_results['hits']['hits'], 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i, rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])  # 'docid'만 사용
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response

In [18]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 5:
            #   break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [19]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/data/ephemeral/home/upstage-ai-final-ir2/HM/data/eval.jsonl", "sample_submission_5.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/tmp/ipykernel_741802/2743210528.py:51: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index='test', body=body, size=size)
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 